In [1]:
import pandas as pd
import numpy as np

se importan las tablas tanto de application train como test

In [2]:
data_train = pd.read_csv('./data/application_train.csv')
data_test = pd.read_csv('./data/application_test.csv')

como se puede ver se pressenta un desbalance considerable, por ello se tomarán todas las etiquetas de 1 y de cero se tomarán la misma cantidad de 1 + 0.20 del tamaño de las de cero, esto para darle un peso mayor a esta clase

In [3]:
data_train_1 = data_train.loc[data_train.loc[data_train.index,'TARGET']==1]
data_train_0 = data_train.loc[data_train.loc[data_train.index,'TARGET']==0]

print('---- Tamaños originales ----')
print('train target 0: ', data_train_0.shape)
print('train target 1: ', data_train_1.shape)

#data_train_0 = data_train_0[:len(data_train_1)+int(1*len(data_train_1))]

print('---- Tamaños nuevos  ----')
print('train target 0: ', data_train_0.shape)
print('train target 1: ', data_train_1.shape)

---- Tamaños originales ----
train target 0:  (282686, 122)
train target 1:  (24825, 122)
---- Tamaños nuevos  ----
train target 0:  (282686, 122)
train target 1:  (24825, 122)


ahora se concatenan para formar el conjunto de datos de entrenamiento

In [4]:
data_train = pd.concat((data_train_0,data_train_1)).sort_index()
data_train.shape, data_test.shape

((307511, 122), (48744, 121))

se indexan los dataframes con el id de usuraio

In [5]:
data_train = data_train.set_index(data_train.SK_ID_CURR)
data_test = data_test.set_index(data_test.SK_ID_CURR)

data_train = data_train.drop('SK_ID_CURR',axis=1)
data_test = data_test.drop('SK_ID_CURR',axis=1)

ahora se procede a cargar la tabla de previous application para concatenarla con los datos train y test

In [6]:
pre_app = pd.read_csv('./data/previous_application.csv')

In [7]:
pre_app = pre_app.set_index(pre_app.SK_ID_CURR.values)
pre_app = pre_app.drop('SK_ID_CURR',axis=1)

se codifican los datos, ya que algunos son de tipo object

In [8]:
%run ./code/label_encoder.py
pre_app = label_encoder(pre_app)

debido a que un cliente puede tener varias aplicaciones previas, se toma la media de dichas aplicaciones para poder hacer el merge entre las tablas

In [9]:
pre_app = pre_app.groupby(pre_app.index).mean()

In [10]:
pre_app = pre_app.drop('SK_ID_PREV',axis=1)

In [11]:
data_train_merge = pd.merge(data_train, pre_app, left_index=True, right_index=True, how='left')
data_test_merge = pd.merge(data_test, pre_app, left_index=True, right_index=True, how='left')

data_train_merge.shape, data_test_merge.shape

((307511, 156), (48744, 155))

ahora se procederá a cargar la tabla de bureau

In [12]:
bureau = pd.read_csv('./data/bureau.csv')
bureau.shape

(1716428, 17)

In [13]:
bureau = bureau.set_index(bureau.columns[0])
bureau = bureau.drop('SK_ID_BUREAU', axis=1)
bureau.head()

,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
SK_ID_CURR,,,,,,,,,,,,,,,
215354,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
215354,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
215354,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
215354,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
215354,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


se procede a codificar y agrupar por la media

In [14]:
bureau = label_encoder(bureau)
bureau = bureau.groupby(bureau.index).mean()
bureau.shape

(305811, 15)

In [15]:
bureau.head()

,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
SK_ID_CURR,,,,,,,,,,,,,,,
100001,1.142857,0.0,-735.000000,0.0,82.428571,-825.500000,NaN,0.0,207623.571429,85240.928571,0.00000,0.0,3.000000,-93.142857,3545.357143
100002,1.500000,0.0,-874.000000,0.0,-349.000000,-697.500000,1681.029,0.0,108131.945625,49156.200000,7997.14125,0.0,3.500000,-499.875000,0.000000
100003,1.500000,0.0,-1400.750000,0.0,-544.500000,-1097.333333,0.000,0.0,254350.125000,0.000000,202500.00000,0.0,3.500000,-816.000000,NaN
100004,2.000000,0.0,-867.000000,0.0,-488.500000,-532.500000,0.000,0.0,94518.900000,0.000000,0.00000,0.0,3.000000,-532.000000,NaN
100005,0.666667,0.0,-190.666667,0.0,439.333333,-123.000000,0.000,0.0,219042.000000,189469.500000,0.00000,0.0,3.333333,-54.333333,1420.500000


se procede a juntar con las demas tablas

In [16]:
data_train_merge = pd.merge(data_train_merge, bureau, left_index=True, right_index=True, how='left')
data_test_merge = pd.merge(data_test_merge, bureau, left_index=True, right_index=True, how='left')

data_train_merge.shape, data_test_merge.shape

((307511, 171), (48744, 170))

ahora se procede a cargar la tabla de credict card balance..

In [17]:
credict_cb = pd.read_csv('./data/credit_card_balance.csv')

In [18]:
credict_cb = credict_cb.set_index(credict_cb.columns[1])
credcit_cb = credict_cb.drop('SK_ID_PREV',axis=1)
print(credict_cb.shape)

(3840312, 22)


se codifica y se le saca la media ya que hay clientes con mas de un registro

In [19]:
credict_cb = label_encoder(credict_cb)
credict_cb = credict_cb.groupby(credict_cb.index).mean()
credict_cb.shape

(103558, 22)

se hace el merge con los datos anteriores...

In [20]:
data_train_merge = pd.merge(data_train_merge, credict_cb, left_index=True, right_index=True, how='left')
data_test_merge = pd.merge(data_test_merge, credict_cb, left_index=True, right_index=True, how='left')

data_train_merge.shape, data_test_merge.shape

((307511, 193), (48744, 192))

ahora se procede a cargar el posh cash balance...

In [21]:
posh_cb = pd.read_csv('./data/POS_CASH_balance.csv')
posh_cb = posh_cb.set_index(posh_cb.columns[1])
posh_cb = posh_cb.drop('SK_ID_PREV',axis=1)
posh_cb.shape

(10001358, 6)

se codifica y se saca la media

In [22]:
posh_cb = label_encoder(posh_cb)
posh_cb = posh_cb.groupby(posh_cb.index).mean()
posh_cb.shape

(337252, 6)

se unen con los datos anteriores...

In [23]:
data_train_merge = pd.merge(data_train_merge, posh_cb, left_index=True, right_index=True, how='left')
data_test_merge = pd.merge(data_test_merge, posh_cb, left_index=True, right_index=True, how='left')

data_train_merge.shape, data_test_merge.shape

((307511, 199), (48744, 198))

In [24]:
#lo mismo para el data_train
col_mean_train = (data_train_merge.loc[:,data_train_merge.count()<data_train_merge.shape[0]].min())
dict_col_mean_train = {}
for i in range(len(col_mean_train)):
    dict_col_mean_train[col_mean_train.index[i]] = col_mean_train[i]
    
#lo mismo para el data_test
col_mean_test = (data_test_merge.loc[:,data_test_merge.count()<data_test_merge.shape[0]].min())
dict_col_mean_test = {}
for i in range(len(col_mean_test)):
    dict_col_mean_test[col_mean_test.index[i]] = col_mean_test[i]

In [25]:
col_mean_train

AMT_ANNUITY_x                   1.615500e+03
AMT_GOODS_PRICE_x               4.050000e+04
OWN_CAR_AGE                     0.000000e+00
CNT_FAM_MEMBERS                 1.000000e+00
EXT_SOURCE_1                    1.456813e-02
EXT_SOURCE_2                    8.173617e-08
EXT_SOURCE_3                    5.272652e-04
APARTMENTS_AVG                  0.000000e+00
BASEMENTAREA_AVG                0.000000e+00
YEARS_BEGINEXPLUATATION_AVG     0.000000e+00
YEARS_BUILD_AVG                 0.000000e+00
COMMONAREA_AVG                  0.000000e+00
ELEVATORS_AVG                   0.000000e+00
ENTRANCES_AVG                   0.000000e+00
FLOORSMAX_AVG                   0.000000e+00
FLOORSMIN_AVG                   0.000000e+00
LANDAREA_AVG                    0.000000e+00
LIVINGAPARTMENTS_AVG            0.000000e+00
LIVINGAREA_AVG                  0.000000e+00
NONLIVINGAPARTMENTS_AVG         0.000000e+00
NONLIVINGAREA_AVG               0.000000e+00
APARTMENTS_MODE                 0.000000e+00
BASEMENTAR

In [26]:
data_train_merge_t = data_train_merge.fillna(value=dict_col_mean_train)
data_test_merge_t = data_test_merge.fillna(value=dict_col_mean_test)

In [27]:
X_train = data_train_merge_t[[col for col in data_train_merge_t.columns if col!='TARGET']]
y_train = data_train_merge_t.TARGET

X_test = data_test_merge_t

X_train.shape, y_train.shape, X_test.shape

((307511, 198), (307511,), (48744, 198))

falta codificar los datos de test, train

In [28]:
X_train = label_encoder(X_train)
X_test = label_encoder(X_test)

/home/edwin/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [31]:
X_train.head()

,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,NAME_TYPE_SUITE_x,...,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS_y,SK_DPD_x,SK_DPD_DEF_x,MONTHS_BALANCE_y,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD_y,SK_DPD_DEF_y
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,6,...,0.0,0.0,0.0,0.0,-10.000000,24.000000,15.000000,0.000000,0.0,0.0
100003,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,1,...,0.0,0.0,0.0,0.0,-43.785714,10.107143,5.785714,0.285714,0.0,0.0
100004,1,1,1,1,0,67500.0,135000.0,6750.0,135000.0,6,...,0.0,0.0,0.0,0.0,-25.500000,3.750000,2.250000,1.000000,0.0,0.0
100006,0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,6,...,0.0,0.0,0.0,0.0,-9.619048,12.000000,8.650000,0.666667,0.0,0.0
100007,0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,6,...,0.0,0.0,0.0,0.0,-33.636364,15.333333,8.969697,0.287879,0.0,0.0


In [29]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(learning_rate=.5, n_estimators=150, verbose=1)
GBC.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           0.5331           12.44m
         2           0.5238           11.96m
         3           0.5176           11.58m
         4           0.5127           11.67m
         5           0.5095           11.47m
         6           0.5063           11.56m
         7           0.5044           11.53m
         8           0.5024           11.45m
         9           0.5012           11.32m
        10           0.4997           11.22m
        20           0.4894           10.08m
        30           0.4841            9.10m
        40           0.4805            8.17m
        50           0.4775            7.41m
        60           0.4750            6.65m
        70           0.4732            5.86m
        80           0.4714            5.11m
        90           0.4699            4.34m
       100           0.4681            3.61m


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.5, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=150,
              presort='auto', random_state=None, subsample=1.0, verbose=1,
              warm_start=False)

In [30]:
GBC.score(X_train, y_train)

0.92244179882996058

In [31]:
predict = GBC.predict_proba(X_test)[:,1]

In [32]:
predict

array([ 0.05298438,  0.10376975,  0.02468615, ...,  0.02355627,
        0.07138417,  0.18053324])

### Mejor Score

In [33]:
#sin correlación de datos
result = pd.DataFrame(data=predict, index=data_test_merge.index, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_51.csv')

In [34]:
sum(X_train.isnull().sum().values)

0

In [30]:
import lightgbm as lgb

clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

In [31]:
clf.score(X_train, y_train)

0.9212093225933381

In [33]:
predict = clf.predict_proba(X_test)[:,1]

In [34]:
predict

array([ 0.04793516,  0.10525869,  0.02678388, ...,  0.0217018 ,
        0.05970344,  0.22352771])

In [35]:
#sin correlación de datos
result = pd.DataFrame(data=predict, index=data_test_merge.index, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_52.csv')

agregando columnas

In [39]:
X_train['DAYS_BIRTH'] = np.abs(X_train['DAYS_BIRTH'])
X_test['DAYS_BIRTH'] = np.abs(X_test['DAYS_BIRTH'])

/home/edwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [43]:
import lightgbm as lgb

clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

In [44]:
clf.score(X_train, y_train)

0.92114103235331424

In [45]:
predict = clf.predict_proba(X_test)[:,1]

In [46]:
predict

array([ 0.03858384,  0.10711838,  0.02701485, ...,  0.02260058,
        0.05691228,  0.2396377 ])

In [47]:
#sin correlación de datos
result = pd.DataFrame(data=predict, index=data_test_merge.index, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_53.csv')

In [50]:
(X_train['OWN_CAR_AGE'] - (X_train['OWN_CAR_AGE'] - (X_train['DAYS_BIRTH']/365))).head()

SK_ID_CURR
100002   -25.920548
100003   -45.931507
100004   -52.180822
100006   -52.068493
100007   -54.608219
dtype: float64

ahora si se prueba agregando unas columnas

In [55]:
#X_train['NEW_CAR_TO_BIRTH_RATIO'] = X_train['OWN_CAR_AGE'] / X_train['DAYS_BIRTH']
X_train['DAYS_FOR_NEW_CAR'] = X_train['OWN_CAR_AGE'] - (X_train['OWN_CAR_AGE'] - (X_train['DAYS_BIRTH']/365))
X_train['NEW_CAR_TO_BIRTH_RATIO'] = X_train['OWN_CAR_AGE'] / (X_train['DAYS_BIRTH']/365)
X_train['NEW_CAR_TO_EMPLOY_RATIO'] = X_train['OWN_CAR_AGE'] / X_train['DAYS_EMPLOYED']
X_train['NEW_PHONE_TO_BIRTH_RATIO'] = X_train['DAYS_LAST_PHONE_CHANGE'] / X_train['DAYS_BIRTH']
X_train['NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER'] = X_train['DAYS_LAST_PHONE_CHANGE'] / X_train['DAYS_EMPLOYED']
X_train['NEW_CREDIT_TO_INCOME_RATIO'] = X_train['AMT_CREDIT_x'] / X_train['AMT_INCOME_TOTAL']

X_test['DAYS_FOR_NEW_CAR'] = X_train['OWN_CAR_AGE'] - (X_train['OWN_CAR_AGE'] - (X_train['DAYS_BIRTH']/365))
X_test['NEW_CAR_TO_BIRTH_RATIO'] = X_train['OWN_CAR_AGE'] / (X_train['DAYS_BIRTH']/365)
#X_test['NEW_CAR_TO_BIRTH_RATIO'] = X_test['OWN_CAR_AGE'] / X_test['DAYS_BIRTH']
X_test['NEW_CAR_TO_EMPLOY_RATIO'] = X_test['OWN_CAR_AGE'] / X_test['DAYS_EMPLOYED']
X_test['NEW_PHONE_TO_BIRTH_RATIO'] = X_test['DAYS_LAST_PHONE_CHANGE'] / X_test['DAYS_BIRTH']
X_test['NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER'] = X_test['DAYS_LAST_PHONE_CHANGE'] / X_test['DAYS_EMPLOYED']
X_test['NEW_CREDIT_TO_INCOME_RATIO'] = X_test['AMT_CREDIT_x'] / X_test['AMT_INCOME_TOTAL']

In [52]:
import lightgbm as lgb

clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

In [53]:
clf.score(X_train, y_train)

0.92115404001808066

In [56]:
predict = clf.predict_proba(X_test)[:,1]

In [57]:
predict

array([ 0.03684887,  0.11696687,  0.02837525, ...,  0.03618421,
        0.04781164,  0.24295651])

In [58]:
#sin correlación de datos
result = pd.DataFrame(data=predict, index=data_test_merge.index, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_56.csv')

In [59]:
dropcolum=['FLAG_DOCUMENT_2','FLAG_DOCUMENT_4',
           'FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7',
           'FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10', 
           'FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13',
           'FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16',
           'FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19',
           'FLAG_DOCUMENT_20','FLAG_DOCUMENT_21']

X_train= X_train.drop(dropcolum,axis=1)
X_test= X_test.drop(dropcolum,axis=1)

In [60]:
import lightgbm as lgb

clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

In [61]:
clf.score(X_train, y_train)

0.92121582642572131

In [62]:
predict = clf.predict_proba(X_test)[:,1]

In [63]:
predict

array([ 0.07293151,  0.12046468,  0.02528744, ...,  0.02687497,
        0.05132805,  0.23377769])

In [64]:
#sin correlación de datos
result = pd.DataFrame(data=predict, index=data_test_merge.index, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_57.csv')

### no hay que eliminar las banderas, tratar de mirar que nuevas columnas sacar, hacer un detallado bien de las columnas, tratar de sacar razónes